In [ ]:
# !pip install mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 31.5 MB 1.3 MB/s 


In [ ]:
# !pip install opencv-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [38]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import math
import time

from PIL import Image
import cv2
import mediapipe as mp
from mediapipe.python.solutions import pose as mp_pose

from google.colab.patches import cv2_imshow

import torch


In [39]:
# Model
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

yolo_model.classes = [0]

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-11-5 Python-3.7.15 torch-1.12.1+cu113 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [40]:
def findposition(results, img, draw=True):
    lmList = []
    # if results are available
    if results.pose_landmarks:
        for lm_id, lm in enumerate(results.pose_landmarks.landmark):
            h, w, c = img.shape
            # print(lm_id, lm)
            # lm contains id, x,y,z, visibility score.
            # x,y,z are ratios, to get the pixel values from landmark values,
            # need to multiply landmark ratios (x or y) with image shape (w or h)
            cx, cy = int(lm.x * w), int(lm.y * h)
            # can choose what fields to be stored. Here just include all 5 available fields
            lmList.append([lm_id, cx, cy, round(lm.z, 5), round(lm.visibility, 7)])
            if draw:
                # overlay on existing dots
                cv2.circle(img, (cx, cy), 5, (255, 0, 0), cv2.FILLED)

    return lmList

def findAngle(lmList, img, p1, p2, p3, draw=True):
    # get landmarks
    x1, y1 = lmList[p1][1:3]
    x2, y2 = lmList[p2][1:3]
    x3, y3 = lmList[p3][1:3]

    # calculate angle (convert radians to degrees)
    angle = math.degrees(math.atan2(y3 - y2, x3 - x2) - math.atan2(y1 - y2, x1 - x2))
    if angle < 0:  # if negative angle value, convert to positive
        angle += 360
    # print('Angle:', angle)

    # draw
    if draw:
        cv2.line(img, (x1, y1), (x2, y2), (255, 255, 255), 2)  # white line
        cv2.line(img, (x3, y3), (x2, y2), (255, 255, 255), 2)  # white line
        cv2.circle(img, (x1, y1), 10, (0, 0, 255), cv2.FILLED)
        cv2.circle(img, (x1, y1), 15, (0, 0, 255), 1)
        cv2.circle(img, (x2, y2), 10, (0, 0, 255), cv2.FILLED)
        cv2.circle(img, (x2, y2), 15, (0, 0, 255), 1)
        cv2.circle(img, (x3, y3), 10, (0, 0, 255), cv2.FILLED)
        cv2.circle(img, (x3, y3), 15, (0, 0, 255), 1)
        cv2.putText(img, str(int(angle)), (x2 - 50, y2 + 50), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2)

    return angle

In [41]:
# cap = cv2.VideoCapture('../../../recordings/lego_rover_build_44mins/split/split000.mp4')
# cap = cv2.VideoCapture('../../../recordings/lego_rover_build_44mins/split/split001.mp4')
# cap = cv2.VideoCapture('../../../recordings/lego_rover_build_44mins/split/split002.mp4')
# cap = cv2.VideoCapture('../../../recordings/lego_rover_build_44mins/split/split003.mp4')
# cap = cv2.VideoCapture('../../../recordings/lego_rover_build_44mins/split/split004.mp4')
cap = cv2.VideoCapture('/content/drive/MyDrive/U/phd/work4_openvino/recordings/lego_rover_build_44mins/split10s/split001.mp4')

while cap.isOpened():
    success, frame = cap.read()
    h, w, _ = frame.shape
    size = (w, h)
    print(size)
    break

# webcam
# cap = cv2.VideoCapture(0)

(720, 480)


In [42]:
# save video file as output.avi
out = cv2.VideoWriter('/content/drive/MyDrive/U/phd/work4_openvino/PycharmProjects/projectCV/HumanPoseEstimation/output/output4.avi', 
                     cv2.VideoWriter_fourcc(*"MJPG"), 20, size)

In [43]:

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # recolor
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # make image writable to False to improve prediction performance
    image.flags.writeable = False

    result = yolo_model(image)

    # recolor image back to BGR for rendering
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    print(result.xyxy)

    img_list = []
    MARGIN = 10

    for (xmin, ymin, xmax, ymax, confidence, clas) in result.xyxy[0].tolist():

        count_r = 0
        count_l = 0
        dir_r = 0
        dir_l = 0
        pTime = 0
        frames = []

        with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
            # pose prediction
            image_crop = image[int(ymin)+MARGIN:int(ymax)+MARGIN, int(xmin)+MARGIN:int(xmax)+MARGIN:]
            # results = pose.process(image[int(ymin)+MARGIN:int(ymax)+MARGIN, int(xmin)+MARGIN:int(xmax)+MARGIN:])
            results = pose.process(image_crop)

            # draw landmarks
            mp_drawing.draw_landmarks(image[int(ymin)+MARGIN:int(ymax)+MARGIN, int(xmin)+MARGIN:int(xmax)+MARGIN:],
                                      results.pose_landmarks,
                                      mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                      )
            
            # img = findpose(img, draw=False)
            lmList = findposition(results, image_crop, draw=False)

            if len(lmList) != 0:
                # right arm
                angle_r = findAngle(lmList, image_crop, 12, 14, 16, True)
                # convert to percentage - check the tradeoff between range and accuracy
                # this is also a place to filter out noise poses
                per_r = np.interp(angle_r, (30, 170), (0, 100))  # from () to ()
                # bar_r = np.interp(angle_r, (30, 170), (300, 400)) # use angle
                bar_r = np.interp(per_r, (40, 100), (300, 400))  # use percentage
                print('angle_right_arm {}, percentage {}'.format(angle_r, per_r))

                # TODO check: frontal facing, or sideways facing to the camera
                # by
                # if

                # left arm
                angle_l = findAngle(lmList, image_crop, 11, 13, 15, True)
                # convert to percentage - check tradeoff
                per_l = np.interp(angle_l, (30, 170), (0, 100))  # from () to ()
                # bar_l = np.interp(angle_l, (30, 170), (300, 400)) # use angle
                bar_l = np.interp(per_l, (40, 100), (300, 400))  # use percentage
                print('angle_left_arm {}, percentage {}'.format(angle_l, per_l))

                # TODO check: scanning (interaction with Viana), or assembling lego (self-work), or browsing lego from table (self-work)
                # scanning: percentage >= angle >= 120 AND duration is shorter than 3 seconds
                # assembling: angle <= 90 AND duration is longer than 3 seconds
                # other self-work: ELSE
                # if per_r >= 50:

                # TODO: replicate same checks as right arm
                color = (255, 0, 0)  # blue

                # right arm
                if per_r >= 80:
                    color = (0, 255, 0)  # green
                    if dir_r == 0:
                        count_r += 0.5
                        dir_r = 1
                if per_r <= 40:
                    color = (255, 0, 0)  # blue
                    if dir_r == 1:
                        count_r += 0.5
                        dir_r = 0

                # left arm
                if per_l >= 80:
                    color = (0, 255, 0)  # green
                    if dir_l == 0:
                        count_l += 0.5
                        dir_l = 1
                if per_l <= 40:
                    color = (255, 0, 0)  # blue
                    if dir_l == 1:
                        count_l += 0.5
                        dir_l = 0

                print(count_r, count_l)

                # use average percentage as bar value
                bar_avg = np.mean((bar_r, bar_l))

                # display arm counts
                cv2.rectangle(image, (300, 25), (400, 50), color, 2)
                cv2.rectangle(image, (300, 25), (int(bar_avg), 50), color, cv2.FILLED)
                cv2.putText(image, 'R {} L {}'.format(count_r, count_l), (50, 50), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0),
                            2)

                # display fps
                cTime = time.time()
                fps = 1 / (cTime - pTime)
                pTime = cTime
                cv2.putText(image, 'FPS {}'.format(str(int(fps))), (50, 90), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2)


            img_list.append(image[int(ymin):int(ymax), int(xmin):int(xmax):])

        # cv2.imshow('Image', image)
        # cv2_imshow(image)

        # write to video file
        out.write(image)

        #
        # cv2.imshow('Activity recognition', image)
        if cv2.waitKey(0) & 0xFF == ord('q'):
            break

cap.release()
out.release()
cv2.destroyAllWindows()